# Лабораторная работа 7

В работе использовался пакет `ipython-sql` и образ MADlib в докере
После того как собрали докер с MADlib и PostgreSQL, установим MADlib в нашу БД. 
```
src/bin/madpack -p postgres -c postgres/postgres@localhost:5432/lab7 install
```


In [21]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
# PostgreSQL local
%sql postgresql://postgres@localhost:5432/lab7

## Загрузка данных
Создаем таблицу по заданию

In [23]:
%%sql
DROP TABLE IF EXISTS survey;

CREATE TABLE survey(
    mydepv INTEGER, 
    price INTEGER, 
    income INTEGER, 
    age INTEGER
    );

 * postgresql://postgres@localhost:5432/lab7
Done.
Done.


[]

Загружаем данные из CSV


In [24]:
%%sql
COPY survey(mydepv, price, income, age)
FROM '/userdata/survey.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres@localhost:5432/lab7
750 rows affected.


[]

## Добавляем столбцы
Добавляем `price20` (1 для 20\$ 0 иначе) и `price30` (1 для 30\$ 0 иначе)

In [25]:
%%sql
ALTER TABLE survey ADD COLUMN price20 INTEGER;
ALTER TABLE survey ADD COLUMN price30 INTEGER;

 * postgresql://postgres@localhost:5432/lab7
Done.
Done.


[]

In [26]:
%%sql
UPDATE survey SET price20 = 1 WHERE price = 20;
UPDATE survey SET price20 = 0 WHERE price != 20;
UPDATE survey SET price30 = 1 WHERE price = 30;
UPDATE survey SET price30 = 0 WHERE price != 30;

 * postgresql://postgres@localhost:5432/lab7
250 rows affected.
500 rows affected.
250 rows affected.
500 rows affected.


[]

Создадим таблицу в таком формате: зависимая переменная, массив независимых переменных

In [27]:
%%sql
DROP TABLE IF EXISTS survey2;
CREATE TABLE survey2 (
    mydepv BOOLEAN,
    indepv INTEGER[]
);

 * postgresql://postgres@localhost:5432/lab7
Done.
Done.


[]

Вставляем значения

In [28]:
%%sql
INSERT INTO survey2
SELECT survey.mydepv::BOOL, 
ARRAY[1, survey.income, survey.age, survey.price20, survey.price30] AS arr FROM survey;

 * postgresql://postgres@localhost:5432/lab7
750 rows affected.


[]

Выведем первые 5

In [29]:
%%sql 
SELECT * FROM survey2 LIMIT 5;

 * postgresql://postgres@localhost:5432/lab7
5 rows affected.


mydepv,indepv
True,"[1, 59, 55, 0, 1]"
False,"[1, 24, 37, 0, 1]"
False,"[1, 76, 43, 0, 1]"
False,"[1, 45, 32, 0, 1]"
False,"[1, 21, 46, 0, 1]"


## Логистическая регрессия в MADlib

In [30]:
%%sql
DROP TABLE IF EXISTS survey_logregr, survey_logregr_summary;
SELECT madlib.logregr_train('survey2',
              'survey_logregr',
              'mydepv',
              'indepv');

 * postgresql://postgres@localhost:5432/lab7
Done.
1 rows affected.


logregr_train
""


### Выведем summary

In [31]:
%%sql
SELECT unnest(array['intercept',
                    'income', 
                    'age', 
                    'price20', 
                    'price30']) AS attribute,
       unnest(coef) AS coefficient,
       unnest(std_err) AS standard_error,
       unnest(z_stats) AS z_stat,
       unnest(p_values) AS pvalue,
       unnest(odds_ratios) AS odds_ratio
    FROM survey_logregr;

 * postgresql://postgres@localhost:5432/lab7
5 rows affected.


attribute,coefficient,standard_error,z_stat,pvalue,odds_ratio
intercept,-6.02116057636787,0.532440923644448,-11.3085983983993,1.18974837145447e-29,0.00242685141358459
income,0.128759374924942,0.00923035837106702,13.9495531753723,3.16582525170149e-44,1.13741640032794
age,0.0350637796423411,0.0117900836733219,2.97400600486677,0.00293939322817078,1.03568576236067
price20,-0.744177494951807,0.264387873296036,-2.81471871487294,0.00488199444444116,0.475124931924531
price30,-2.21028046675832,0.311075548871486,-7.10528511410408,1.20074395698392e-12,0.109669885444447


### Предсказывания

In [32]:
%%sql
DROP TABLE IF EXISTS survey3;
CREATE TABLE survey3(
    mydepv INTEGER, 
    price INTEGER, 
    income INTEGER, 
    age INTEGER,
    price20 INTEGER,
    price30 INTEGER,
    odds_ratio REAL, 
    prediction REAL
    );

 * postgresql://postgres@localhost:5432/lab7
Done.
Done.


[]

Вставим значения во вторую таблицу с использованием функции MADlib. Для odds_ratio будем использовать коэффициенты (API уже изменен). 

In [33]:
%%sql
INSERT INTO survey3
SELECT mydepv,
        price, 
        income, 
        age, 
        price20, 
        price30,
        (coef[1] + 
        coef[2] * income + 
        coef[3] * age + 
        coef[4] * price20 + 
        coef[5] * price30) AS odds_ratio,
        madlib.logregr_predict_prob(coef, ARRAY[1, 
                                            income, 
                                            age, 
                                            price20, 
                                            price30]) AS prediction
FROM survey_logregr, survey AS s;

 * postgresql://postgres@localhost:5432/lab7
750 rows affected.


[]

Делаем предсказание для 58к 25 лет и 20\$

In [34]:

%%sql
select madlib.logregr_predict_prob(coef,
                ARRAY[1, 58, 25, 1, 0]),
madlib.logregr_predict(coef,
                ARRAY[1, 58, 25, 1, 0])
from survey_logregr;


 * postgresql://postgres@localhost:5432/lab7
1 rows affected.


logregr_predict_prob,logregr_predict
0.829105381494624,True


Делаем предсказание для 649 долларов, 60 лет и 10 долларов

In [35]:

%%sql
select madlib.logregr_predict_prob(coef,
                ARRAY[1, 0.649, 60, 0, 0]),
madlib.logregr_predict(coef,
                ARRAY[1, 0.649, 60, 0, 0])
from survey_logregr;


 * postgresql://postgres@localhost:5432/lab7
1 rows affected.


logregr_predict_prob,logregr_predict
0.0211700814830798,False


Проверим, что наши предсказания совпадают

In [36]:
%%sql
SELECT SUM(mydepv) AS mydepv, SUM(prediction)::INTEGER AS prediction FROM survey3;

 * postgresql://postgres@localhost:5432/lab7
1 rows affected.


mydepv,prediction
324,324


Выведем результирующую таблицу

In [37]:
%%sql
SELECT * FROM survey3 LIMIT 5;

 * postgresql://postgres@localhost:5432/lab7
5 rows affected.


mydepv,price,income,age,price20,price30,odds_ratio,prediction
1,30,59,55,0,1,1.29387,0.784801
0,30,24,37,0,1,-3.84386,0.0209621
0,30,76,43,0,1,3.06201,0.955298
0,30,45,32,0,1,-1.31523,0.211613
0,30,21,46,0,1,-3.91456,0.0195591
